In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("주식분석")

LangSmith 추적을 시작합니다.
[프로젝트명]
주식분석


In [3]:
#네이버검색

import os
import requests

class NaverSearchAPIWrapper:
    def __init__(self):
        self.client_id = os.getenv("NAVER_CLIENT_ID")
        self.client_secret = os.getenv("NAVER_CLIENT_SECRET")
        self.url = "https://openapi.naver.com/v1/search/webkr.json"

    def run(self, query):
        headers = {
            "X-Naver-Client-Id": self.client_id,
            "X-Naver-Client-Secret": self.client_secret
        }
        params = {
            "query": query,
            "display": 3  # 검색 결과 수
        }
        response = requests.get(self.url, headers=headers, params=params)
        if response.status_code == 200:
            items = response.json().get("items", [])
            results = [item['title'] + " - " + item['link'] for item in items]
            return "\n".join(results)
        else:
            return "검색 실패: 상태 코드 " + str(response.status_code)


In [4]:
from typing import TypedDict, NotRequired
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
from web import search  # 웹 검색을 위한 모듈

# 1. Stock Value Output 구조 정의
class StockValueOutput(TypedDict):
    Net_Income: NotRequired[str]  # 순이익
    Shares_Outstanding: NotRequired[str]  # 발행주식수
    Stock_Price: NotRequired[str]  # 주가
    Book_Value: NotRequired[str]  # 자기자본
    FCF: NotRequired[str]  # 자유현금흐름
    EBIT: NotRequired[str]  # 영업이익
    WACC: NotRequired[str]  # 할인율
    PFCF: NotRequired[str]  # 미래 현금흐름
    Growth_Rate: NotRequired[str]  # 성장률
    DPS: NotRequired[str]  # 주당배당금
    ROE: NotRequired[str]  # 자기자본이익률
    PER: NotRequired[str]  # 주가수익비율
    PBR: NotRequired[str]  # 주가순자산비율
    DCF: NotRequired[str]  # 할인현금흐름
    DDM: NotRequired[str]  # 배당할인모형

# 2. LLM 및 Retriever 설정
stock_value_model = ChatOpenAI(model="gpt-4o-mini")

# 3. 벡터 검색을 위한 Retriever 설정 (FAISS 활용)
embedding_model = OpenAIEmbeddings()
vector_db = FAISS.load_local("faiss_stock_db", embedding_model)  # 로컬 FAISS 벡터 DB 불러오기
retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 5})  # 유사한 문서 5개 검색

# 4. Stock Value Calculation (RAG + 웹 검색 보완)
def stock_value_calculation_node(state: dict):
    user_input = state.get("user_input", "")  # 안전하게 가져오기

    # 1️⃣ RAG 검색 수행
    retrieved_docs = retriever.get_relevant_documents(user_input)
    retrieved_texts = "\n".join([doc.page_content for doc in retrieved_docs])

    # 2️⃣ RAG 결과에서 필요한 데이터 추출
    extracted_data = {}
    missing_fields = [
        "Net_Income", "Shares_Outstanding", "Stock_Price", "Book_Value", "FCF",
        "EBIT", "WACC", "PFCF", "Growth_Rate", "DPS", "ROE", "PER", "PBR", "DCF", "DDM"
    ]

    if retrieved_texts:
        # LLM을 사용하여 검색된 정보에서 필요한 데이터 추출
        messages = [
            {"role": "system", "content": """
            다음은 검색된 관련 정보입니다. 이 정보를 분석하여 아래 항목에 해당하는 값을 추출하세요:
            'Net_Income', 'Shares_Outstanding', 'Stock_Price', 'Book_Value', 'FCF',
            'EBIT', 'WACC', 'PFCF', 'Growth_Rate', 'DPS', 'ROE', 'PER', 'PBR', 'DCF', 'DDM'.
            """},
            {"role": "system", "content": f"검색된 정보:\n{retrieved_texts}"},
            {"role": "user", "content": "위 항목의 값을 추출하여 JSON 형식으로 제공해 주세요."}
        ]

        extracted_data = stock_value_model.invoke(messages)  # LLM을 사용하여 데이터 추출

        # RAG에서 추출된 값이 없는 항목을 필터링하여 웹 검색 목록 생성
        missing_fields = [key for key in missing_fields if not extracted_data.get(key)]

    # 3️⃣ RAG에서 찾지 못한 값만 웹 검색 수행
    if missing_fields:
        print(f"🔍 웹 검색 수행 (필요한 값: {', '.join(missing_fields)})...")
        web_results = search(user_input + " 재무제표 OR 반기보고서 OR 주식가치 관련 정보")

        if web_results:
            retrieved_texts += "\n" + "\n".join(web_results[:3])  # 상위 3개 결과만 사용

            # LLM을 활용하여 웹 검색 결과에서 부족한 데이터 추출
            messages = [
                {"role": "system", "content": f"""
                다음은 추가 웹 검색 결과입니다. 기존 정보와 함께 부족한 데이터를 채워주세요:
                {', '.join(missing_fields)}
                """},
                {"role": "system", "content": f"웹 검색 결과:\n{retrieved_texts}"},
                {"role": "user", "content": "부족한 값들을 JSON 형식으로 제공해 주세요."}
            ]

            web_extracted_data = stock_value_model.invoke(messages)
            extracted_data.update(web_extracted_data)  # 웹 검색에서 얻은 데이터 추가

    # 4️⃣ LLM이 최종 정리하여 답변 생성
    messages = [
        {"role": "system", "content": """
        당신은 유능한 금융 전문가입니다.
        사용자의 입력을 기반으로 주식 가치를 평가하세요.
        검색된 데이터를 참고하여 종합적인 분석을 제공합니다.
        """},
        {"role": "user", "content": user_input},
        {"role": "system", "content": f"다음은 검색된 관련 정보입니다:\n{retrieved_texts}"},
        {"role": "system", "content": f"추출된 데이터:\n{extracted_data}"}
    ]

    response = stock_value_model.invoke(messages)

    print("================================= Stock Value Calculation (RAG + 웹 검색 보완) =================================")
    print(f"입력된 정보: {user_input}")
    print(f"검색된 정보:\n{retrieved_texts}")
    print(f"최종 데이터: {extracted_data}")
    
    return {"stock_value_dict": extracted_data, "final_response": response}

ModuleNotFoundError: No module named 'web'